In [131]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "../../utils/")
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from training_models.classification_models import ClassificationModels
from joblib import dump

In [132]:
def n_samples(df):
    df_high = df[df['target'] == 2]
    df_mid = df[df['target'] == 1]
    df_low = df[df['target'] == 0]
    n=min(len(df_high), len(df_mid), len(df_low))
    return n

In [133]:
name="one_hot"
df_data = pd.read_csv(f"../../data/numerical_rep/{name}_antiviral_homology_90.csv")
df_data.drop(["experimental_characteristics"], axis=1, inplace=True)

In [134]:
model_funcs = {
    "AdaBoost": "instance_ada_boost",
    "RandomForest": "instance_random_forest",
    "SVM": "instance_svc",
    "GradientBoosting": "instance_gradient_boosting",
    "LogisticRegression": "instance_logistic_regresion",
    "XGBoost": "instance_xg_boost",
    "Lgbm": "instance_lgbm",
    "KNN": "instance_neighbors"
}

In [135]:
rename_map = {
    "f1_weighted": "F1-score",
    "recall_weighted": "Recall",
    "precision_weighted": "Precision",
    "accuracy": "Accuracy"
}

In [136]:
all_metrics = []
i=np.random.randint(0, 100)

In [137]:
train_data, val_data = train_test_split(df_data, test_size=0.2, random_state=i)

In [138]:
n_samples_train= n_samples(train_data)
n_samples_val= n_samples(val_data)

In [139]:
undersampler_train = RandomUnderSampler(sampling_strategy={0: n_samples_train, 1: n_samples_train, 2: n_samples_train}, random_state=i)
undersampler_val = RandomUnderSampler(sampling_strategy={0: n_samples_val, 1: n_samples_val, 2: n_samples_val}, random_state=i)

In [ ]:
X_train = train_data.drop('target', axis=1)
y_train = train_data['target']  
X_val = val_data.drop('target', axis=1)
y_val = val_data['target']

In [141]:
X_resampled, y_resampled = undersampler_train.fit_resample(X_train, y_train)
df_resampled_train = pd.concat([pd.DataFrame(X_resampled, columns=X_train.columns), pd.Series(y_resampled, name='target')], axis=1)
X_resampled, y_resampled = undersampler_val.fit_resample(X_val, y_val)
df_resampled_val = pd.concat([pd.DataFrame(X_resampled, columns=X_val.columns), pd.Series(y_resampled, name='target')], axis=1)

/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils

In [142]:
train_values = df_resampled_train.drop(columns="target").values
train_response = df_resampled_train["target"].values

validation_values = df_resampled_val.drop(columns="target").values
validation_response = df_resampled_val["target"].values

In [ ]:
for model_name, method_name in model_funcs.items():
        print(f"Training model: {model_name}")

        clf_model = ClassificationModels(X_train=train_values, X_val=validation_values, y_train=train_response, y_val=validation_response)

        getattr(clf_model, method_name)()
        clf_model.process_model(kfold=True, k=5)

        train_metrics = clf_model.performances["training_metrics"]
        val_metrics = clf_model.performances["validation_metrics"]
        val_metrics.pop("Confusion Matrix", None)
        train_renamed = {rename_map.get(k, k): v for k, v in train_metrics.items()}

        metrics_row = {"iteration": i, "model": model_name}
        for k in rename_map.values():
            metrics_row[f"Train_{k}"] = round(train_renamed[k], 4)
            metrics_row[f"Val_{k}"] = round(val_metrics[k], 4)
        all_metrics.append(metrics_row)

        dump(clf_model.model, f"../../models/{model_name}.joblib")

df_metrics = pd.DataFrame(all_metrics)
df_metrics.to_csv(f"../../models/metrics_{name}.csv", index=False)

/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Entrenando modelo: AdaBoost


/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Entrenando modelo: RandomForest
Entrenando modelo: SVM
Entrenando modelo: GradientBoosting
Entrenando modelo: LogisticRegression
Entrenando modelo: XGBoost


/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jmartin/.miniforge3/envs/ML_Class/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was depreca

AttributeError: 'super' object has no attribute '__sklearn_tags__'

clf_model = ClassificationModels(
    X_train=train_values, X_val=validation_values, y_train=train_response, y_val=validation_response
)

clf_model.instance_linear_svc()
clf_model.process_model(kfold=True, k=5)

train=clf_model.performances["training_metrics"]
valid=clf_model.performances["validation_metrics"]
valid.pop("Confusion Matrix", None)

train_r = {rename_map.get(k, k): v for k, v in train.items()}
df_metrics = pd.DataFrame({
    "Training": train_r,
    "Validation": valid
})